<a href="https://colab.research.google.com/github/palakagl/NLP/blob/main/Wysdom_ShallowModel_TFIDF_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from nltk.corpus import stopwords

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving personal_assistant_train.csv to personal_assistant_train.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['personal_assistant_train.csv']))

In [ ]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       8884 non-null   int64 
 1   message  8884 non-null   object
 2   label    8884 non-null   object
dtypes: int64(1), object(2)
memory usage: 208.3+ KB
None
      id                                            message             label
0  10480                           start the robotic vacuum      iot_cleaning
1   4210          i am sorry but i think you are not right.    general_negate
2   2443  what is the time difference between california...  datetime_convert
3   5869                              tell me my list names       lists_query
4   2801                 beep when i get an email from john       email_query


In [ ]:
s = df.message
for i in range(len(s)):
  #Removes anything thats not word or number
  s[i] = re.sub(r'\W',' ',s[i])
  #Remove single alphabets
  s[i] = re.sub(r'\s+[a-zA-Z]\s+',' ',s[i])
  #Remove digits
  s[i] = re.sub(r'\d',' ',s[i])
  #Remove space from first character
  s[i] = re.sub(r'^\s',' ',s[i])
  #Remove space from last character
  s[i] = re.sub(r'\s$',' ',s[i])
  #Remove extra spaces
  s[i] = re.sub(r'\s+',' ',s[i])

df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: Sett

,id,message,label
0,10480,start the robotic vacuum,iot_cleaning
1,4210,i am sorry but think you are not right,general_negate
2,2443,what is the time difference between california...,datetime_convert
3,5869,tell me my list names,lists_query
4,2801,beep when get an email from john,email_query
5,7384,find me the local npr station,play_radio
6,3244,yes answer was confirmed,general_affirm
7,3247,answer is confirmed,general_affirm
8,5220,i would love less bright up room,iot_hue_lightdim
9,6400,i ve heard this riff before what song stole th...,music_query


In [ ]:
X = df['message']
y = df['label']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(7107,)
(7107,)
(1777,)
(1777,)


In [ ]:
'''
# Creating the BOW model
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()


# Creating the Tf-Idf Model
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()
'''

In [ ]:
#vectorizer = CountVectorizer(min_df=.01, max_df=.8, ngram_range=[1,3], max_features=550, stop_words='english')
vectorizer = TfidfVectorizer(max_features = 700, min_df = .01, max_df = 0.95, stop_words = stopwords.words('english'))
pipe = Pipeline([('vec', vectorizer),  ('clf', RandomForestClassifier(random_state=223))])

pipe.fit(X_train, y_train)

Pipeline(steps=[('vec',
                 TfidfVectorizer(max_df=0.95, max_features=700, min_df=0.01,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('clf', RandomForestClassifier(random_state=223))])

In [ ]:
pred_val = pipe.predict(X_val)
print(confusion_matrix(y_val, pred_val))
print(classification_report(y_val, pred_val))

[[22  0  7 ...  0  0  0]
 [ 5 12  6 ...  0  0  0]
 [ 3  1 15 ...  0  0  0]
 ...
 [ 0  0  0 ... 24  0  0]
 [ 0  0  0 ...  0 27  0]
 [ 0  0  0 ...  0  0 10]]
                          precision    recall  f1-score   support

             alarm_query       0.73      0.76      0.75        29
            alarm_remove       0.92      0.52      0.67        23
               alarm_set       0.54      0.52      0.53        29
       audio_volume_down       0.38      0.42      0.40        12
       audio_volume_mute       0.00      0.00      0.00        23
         audio_volume_up       0.62      0.21      0.31        24
          calendar_query       0.33      0.06      0.11        31
         calendar_remove       0.61      0.74      0.67        31
            calendar_set       0.46      0.32      0.38        34
          cooking_recipe       0.18      0.26      0.21        35
        datetime_convert       0.83      0.38      0.53        13
          datetime_query       0.50      0.43      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
ari = adjusted_rand_score(y_val, pred_val)
ami = adjusted_mutual_info_score(y_val, pred_val, average_method='arithmetic')

print("ARI: {}".format(ari))
print("AMI: {}".format(ami))

ARI: 0.1218946107716465
AMI: 0.45469029276961664


## Kaggle Predictions

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving input_data.csv to input_data.csv


In [ ]:
df_test = pd.read_csv(io.BytesIO(uploaded['input_data.csv']))

In [ ]:
#df_test = pd.read_csv('input_data.csv')
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2221 non-null   int64 
 1   message  2221 non-null   object
dtypes: int64(1), object(1)
memory usage: 34.8+ KB


,id,message
0,10873,list the current price of microsoft shares
1,8491,where is the nearest smoke shop
2,4549,please start over.
3,5948,read my lists to me
4,1663,mute volume now


In [ ]:
pred_test = pipe.predict(df_test['message'])
pred_test

array(['lists_query', 'general_affirm', 'iot_cleaning', ...,
       'iot_hue_lightdim', 'email_sendemail', 'general_affirm'],
      dtype=object)

In [ ]:

my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
print(my_submission.head())

      Id              label
0  10873        lists_query
1   8491     general_affirm
2   4549       iot_cleaning
3   5948     general_affirm
4   1663  audio_volume_down


In [ ]:
# NOTE: after saving the CSV file, be sure to zip the file before submitting to the competition website!
my_submission.to_csv('Answers_RF.csv', index=False)